# Get Data

Note: data is the three scale channel features

In [ ]:
import gc
import h5py
import numpy as np
load_path='F:/NISTdata_H/Active Mobile/AAPlantD1_2GHz_TX1_hpol_run4_pp.mat'
load_mat=h5py.File(load_path,'r')
IQdata=load_mat['IQdata'][3600:21600,0:256].view('complex')
locations=60
records=300
# load_path='F:/NISTdata_H/Active Mobile/AAPlantD3_2GHz_TX2b_vpol_run32_pp.mat'  # 3900:39900 120
# load_mat=h5py.File(load_path,'r')
# IQdata=load_mat['IQdata'][3900:21900,0:256].view('complex')
# locations=60
# records=300
# load_path='F:/NISTdata_H/Active Mobile/AAPlantD3_2GHz_TX2b_hpol_run34_pp.mat'  # 3900:39900 120
# load_mat=h5py.File(load_path,'r')
# IQdata=load_mat['IQdata'][2100:18000,0:256].view('complex')
# locations=53
# records=300  
# load_path='F:/NISTdata_H/Active Mobile/AAPlantD1_2GHz_TX1_vpol_run3_pp.mat'   # 6600:42600  120
# load_mat=h5py.File(load_path,'r')
# IQdata=load_mat['IQdata'][6600:24600,0:256].view('complex')
# locations=60
# records=300

# Hyper-parameters

In [ ]:
# training parameters
seed = 0                        # random seed
batch_size = 32                # batch size
num_epoch = 30                  # the number of training epoch
patience = 10                  # If no improvement in 'patience' epochs, early stop
learning_rate = 0.0001         # learning rate
model_path = './model.ckpt'     # the path where the checkpoint will be saved

# model parameters
k=1  #H为k+1个h
nf=1 #参考CSI的数量+1
index=[0,1]
d=2  #eve在alice后面d-1个位置处尾随
input_channels=4
len_sequence = 256                  # the input dim of the model, you should not change the value
output_dim=2
mu=0.4
std=0.1
sw=0.45

# creat a sequence

In [ ]:
import numpy as np
from numpy import fft
def convert_seq(h):
    a=np.abs(h)
    b=np.angle(h)
    c=np.abs(fft.fft(h))
    d=np.angle(fft.fft(h))
    seq=np.concatenate((a,b,c,d),axis=0)
    return seq

# creatH

In [ ]:
def creatHSeq_Alice(idx, k, nf):
    H=np.zeros((k+nf,len_sequence)).astype(complex)
    H=IQdata[idx-nf:idx+k]
    H=H[index]
    return  H.flatten()
def creatHSeq_Eve(idx, k, nf, d):
    H=np.zeros((k+nf,len_sequence)).astype(complex)
    H[nf:k+nf,:]=IQdata[idx-d*records:idx+k-d*records]
    H[0:nf,:]=IQdata[idx-nf:idx]
    H=H[index]
    return  H.flatten()

# Preprocess data 

In [ ]:
recs=records*d
num_dataall=locations*records-recs-10
dataall_HimagA=np.zeros((num_dataall,len_sequence*(k+nf)*input_channels)).astype(float)
dataall_HimagE=np.zeros((num_dataall,len_sequence*(k+nf)*input_channels)).astype(float)
for i in range(recs,locations*records-10):
    dataall_HimagA[i-recs,:]=convert_seq(creatHSeq_Alice(i,k=k,nf=nf))
    dataall_HimagE[i-recs,:]=convert_seq(creatHSeq_Eve(i,k=k,nf=nf,d=d))

dataall_TW=np.concatenate((dataall_HimagA,dataall_HimagE),axis=0)
location=[i for i in range(2)]
test_data=dataall_TW
test_lab=[val for val in location for i in range(int(num_dataall))]
train_data=dataall_TW[0:int(num_dataall*2):2]
train_label=[val for val in location for i in range(int(num_dataall/2))]
valid_data=dataall_TW[1:int(num_dataall*2):2]
valid_label=[val for val in location for i in range(int(num_dataall/2))]

del dataall_HimagA, dataall_HimagE, dataall_TW
gc.collect()

# Training

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import joblib

tree = DecisionTreeClassifier(criterion='entropy',
                              min_samples_leaf=10, 
                              max_leaf_nodes=20, 
                              random_state=220)
bagging = BaggingClassifier(tree,
                            n_estimators=19, 
                            random_state=220)

_exp_name = "H_BT_hrun4_sample"   
# ---training---
bagging.fit(train_data, train_label)
tra_label=bagging.predict(train_data)       
acc=accuracy_score(train_label,tra_label)
# Print the information.
print(f"[ Train ] , acc = {acc:.5f}")
joblib.dump(bagging, f"{_exp_name}_best.m") # only save best to prevent output memory exceed error

# Testing

In [ ]:
model_best = joblib.load(f"{_exp_name}_best.m")
tes_label=model_best.predict(test_data)
accuracy=accuracy_score(tes_label,test_lab)
print(accuracy)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style
error_train=np.array(tra_label)-np.array(train_label)
plt.figure(1)
plt.plot(error_train, linestyle="--", label="train")
plt.xlabel('Records')
plt.ylabel('Error')
plt.show
error_test=np.array(tes_label)-np.array(test_lab)
import matplotlib.pyplot as plt
plt.figure(2)
plt.plot(error_test, linestyle="--")
plt.xlabel('Records')
plt.ylabel('Error')
plt.show